In [ ]:
# Import library yang diperlukan
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.tree import plot_tree, export_text
import matplotlib.pyplot as plt

# STEP 1: Membaca Data dari Excel
file_path = "data.xlsx"  # Sesuaikan dengan nama file Anda
data = pd.read_excel(file_path)

# Menampilkan data awal
print("Data Awal:")
print(data.head())


In [ ]:
# STEP 2: Preprocessing Data
# Encoding variabel kategori
label_encoders = {}
for column in ['Kelas', 'Prestasi Akademik (Kelas)']:
    le = LabelEncoder()
    data[column] = le.fit_transform(data[column])
    label_encoders[column] = le

# Menampilkan data setelah encoding
print("\nData Setelah Encoding:")
print(data.head())

# Pisahkan fitur dan target
X = data[['Matematika', 'Bahasa Indonesia', 'Bahasa Inggris', 'Kehadiran (%)', 'Disiplin (Skor)']]
y = data['Prestasi Akademik (Kelas)']

In [ ]:
# Import library untuk penyimpanan model
import joblib

# STEP 3: Membagi Data ke Training dan Testing
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# STEP 4: Membentuk Model Random Forest
rf_model = RandomForestClassifier(n_estimators=100, random_state=42)
rf_model.fit(X_train, y_train)

# Menyimpan model ke file
model_filename = "random_forest_model.pkl"
joblib.dump(rf_model, model_filename)
print(f"Model berhasil disimpan ke file: {model_filename}")


In [ ]:
# Import library yang diperlukan
import joblib
import matplotlib.pyplot as plt
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.tree import plot_tree, export_text

# STEP 5: Memuat Model yang Sudah Ada
model_filename = "random_forest_model.pkl"  # Sesuaikan dengan nama file model yang sudah disimpan
rf_model = joblib.load(model_filename)
print(f"Model berhasil dimuat dari file: {model_filename}")

In [ ]:
# STEP 6: Evaluasi Model
y_pred = rf_model.predict(X_test)

# Menampilkan laporan klasifikasi
print("\nLaporan Klasifikasi:")
print(classification_report(y_test, y_pred))

# Menampilkan confusion matrix
conf_matrix = confusion_matrix(y_test, y_pred)
print("\nConfusion Matrix:")
print(conf_matrix)

In [ ]:
from sklearn.tree import export_text, plot_tree
import matplotlib.pyplot as plt

# Ganti kelas numerik menjadi label deskriptif
class_mapping = {0.0: "Tidak Berprestasi", 1.0: "Cukup Berprestasi", 2.0: "Sangat Berprestasi"}

# Fungsi untuk mengubah kelas numerik menjadi string deskriptif
def map_class_to_label(class_value):
    return class_mapping.get(class_value, "Unknown")

# Ekstrak aturan pohon keputusan dalam format teks
tree_rules = export_text(rf_model.estimators_[0], feature_names=list(X.columns))
print("\nAturan Pohon Keputusan dengan Label Deskriptif:")
# Gantilah kelas numerik menjadi label deskriptif pada aturan pohon
tree_rules_descriptive = tree_rules.replace("class: 0.0", "class: Tidak Berprestasi") \
                                   .replace("class: 1.0", "class: Sedang Berprestasi") \
                                   .replace("class: 2.0", "class: Sangat Berprestasi")
print(tree_rules_descriptive)

# Visualisasi pohon keputusan dengan kelas deskriptif
def plot_tree_with_custom_classes(model, feature_names, class_names):
    plt.figure(figsize=(15, 10))
    
    # Visualisasi pohon keputusan dengan label deskriptif
    plot_tree(model, 
              feature_names=feature_names, 
              class_names=class_names, 
              filled=True, 
              node_ids=True,
              label='all',  # Menambahkan label pada semua simpul
              impurity=False)
    
    plt.title("Visualisasi Pohon Keputusan (Potensi Berprestasi vs Tidak Berpotensi)")
    plt.show()

# Map the class names to more understandable labels for the visualization
class_names_list = ['Tidak Berprestasi', 'Sedang Berprestasi', 'Sangat Berprestasi']

# Menampilkan visualisasi pohon keputusan pertama
plot_tree_with_custom_classes(rf_model.estimators_[0], list(X.columns), class_names_list)


In [ ]:
import matplotlib.pyplot as plt

# Menambahkan prediksi ke data testing
X_test['Prediksi'] = y_pred
X_test['Asli'] = y_test

# Mapping label hasil prediksi dan asli
class_mapping = {0.0: "Tidak Beprestasi", 1.0: "Sedang Beprestasi", 2.0: "Sangat Beprestasi"}

# Mengubah hasil prediksi dan label asli ke dalam bentuk label yang lebih mudah dipahami
X_test['Prediksi Label'] = X_test['Prediksi'].map(class_mapping)
X_test['Asli Label'] = X_test['Asli'].map(class_mapping)

# Plot perbandingan asli vs prediksi
plt.figure(figsize=(10, 6))
X_test['Asli Label'].value_counts().plot(kind='bar', alpha=0.7, color='blue', label='Asli')
X_test['Prediksi Label'].value_counts().plot(kind='bar', alpha=0.5, color='orange', label='Prediksi')
plt.title('Perbandingan Kelas Asli dan Prediksi')
plt.xlabel('Kategori Prestasi Akademik')
plt.ylabel('Jumlah Siswa')
plt.legend()
plt.show()
